In [36]:
import torch
import models
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

In [2]:
args_cuda=True
args_batch_size=64

args_dataset='cifar10'
args_depth = 110
args_arch = 'resnet'

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/jovyan/model_compression/rethinking-network-pruning/cifar/l1-norm-pruning/data.cifar10', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.Pad(4),
                       transforms.RandomCrop(32),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [13]:
# train_set = datasets.CIFAR10('/home/jovyan/model_compression/rethinking-network-pruning/cifar/l1-norm-pruning/data.cifar10', train=True, download=True,)
# loader = torch.utils.data.DataLoader(train_set, batch_size=len(train_set), shuffle=True, **kwargs)
# data = next(iter(loader))

Files already downloaded and verified


In [4]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
#     print(data)
    print(target)
    break

0
tensor([4, 4, 5, 9, 1, 8, 5, 6, 0, 1, 5, 0, 9, 3, 0, 7, 9, 5, 1, 5, 8, 4, 1, 4,
        6, 3, 0, 7, 6, 9, 0, 3, 8, 2, 6, 3, 6, 1, 0, 3, 6, 2, 8, 3, 5, 0, 7, 0,
        1, 9, 8, 6, 2, 5, 3, 3, 6, 0, 6, 7, 2, 5, 5, 8])


In [34]:
data.shape

torch.Size([64, 3, 32, 32])

In [11]:
train_loader.batch_size

64

In [5]:
train_loader.dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /home/jovyan/model_compression/rethinking-network-pruning/cifar/l1-norm-pruning/data.cifar10
    Split: Train
    StandardTransform
Transform: Compose(
               Pad(padding=4, fill=0, padding_mode=constant)
               RandomCrop(size=(32, 32), padding=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )

## model

In [149]:
x =  np.ones((3, 32, 32))
w = np.ones((25, 5, 5))

In [151]:
y = np.matmul(x,w)
y

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 32)

In [70]:
model = models.__dict__[args_arch](dataset=args_dataset, depth=args_depth)

In [110]:
model.parameters

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), strid

In [129]:
for name, param in model.state_dict().items(): #.named_modules():#
    print(name)
    # name: str
    # param: Tensor

conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.num_batches_tracked
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.bn1.running_mean
layer1.2.bn1.running_var
layer1.2.bn1.num_batches_tracked
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.bn2.running_mean
layer1.2.bn2.running_var
layer1.2.bn2.num_batches_tracked
layer1.3.conv1.weight
layer1.3.bn1.weight
laye

In [40]:
model.inplanes #64

64

In [13]:
from compute_flops import print_model_param_flops

In [38]:
from torchsummary import summary
model = model.cuda()
summary(model, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
       BasicBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,

In [54]:
skip = {
    'A': [36],
    'B': [36, 38, 74],
}

prune_prob = {
    'A': [0.5, 0.0, 0.0],
    'B': [0.5, 0.4, 0.3],
}

layer_id = 1
cfg = []
cfg_mask = []
stage = 0
prune_prob_stage = prune_prob['B'][stage]

In [75]:
i=0
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        out_channels = m.weight.data.shape[0]
#         print(m,'-', m.weight.data.shape,'->', out_channels)
        
        #skip---------
#         cfg_mask.append(torch.ones(out_channels)) #一個空的 裡面都存1的matrix
#         cfg.append(out_channels)
#         layer_id += 1
#         print(cfg_mask)
#         print(cfg)
#         print('')
        
        #.clone(): 複製一個完全相同的tensor
        #.cpu(): gpu tensor 轉 cpu tensor
        #.numpy(): tensor 轉 numpy
        weight_copy = m.weight.data.abs().clone().cpu().numpy()
#         print(len(weight_copy)) # k 3*3, 3channel, output channel 16
#         print(weight_copy)
        L1_norm = np.sum(weight_copy, axis=(1,2,3))
#         print(L1_norm)
        num_keep = int(out_channels * (1 - prune_prob_stage))
#         print(num_keep) #8
        arg_max = np.argsort(L1_norm) #x中的元素從小到大排列，提取其對應的index(索引)，然後輸出到y
#         print(arg_max)
        arg_max_rev = arg_max[::-1][:num_keep]
#         print(arg_max_rev)
        mask = torch.zeros(out_channels)
#         print(mask)
        mask[arg_max_rev.tolist()] = 1
        print(mask)
        cfg_mask.append(mask)
        cfg.append(num_keep)
        print(cfg_mask)
        print(cfg)
        i+=1
        if i == 2:
            break

tensor([1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.])
[tensor([0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.]), tensor([1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.])]
[8, 8]
tensor([0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.])
[tensor([0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.]), tensor([1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.]), tensor([0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.])]
[8, 8, 8]


           p  p  p  p  p
            o  o  o  o  o
            s  s  s  s  s

     dim 2  0  1  2  3  4

            |  |  |  |  |
  dim 0     ↓  ↓  ↓  ↓  ↓
  ----> [[[ 0  1  2  3  4]   <---- dim 1, pos 0
  pos 0   [ 5  6  7  8  9]   <---- dim 1, pos 1
          [10 11 12 13 14]]  <---- dim 1, pos 2
  dim 0
  ---->  [[15 16 17 18 19]   <---- dim 1, pos 0
  pos 1   [20 21 22 23 24]   <---- dim 1, pos 1
          [25 26 27 28 29]]] <---- dim 1, pos 2
            ↑  ↑  ↑  ↑  ↑
            |  |  |  |  |

     dim 2  p  p  p  p  p
            o  o  o  o  o
            s  s  s  s  s

            0  1  2  3  4


In [60]:
args_refine = '/home/jovyan/model_compression/rethinking-network-pruning/cifar/l1-norm-pruning/logs/pruned.pth.tar'

In [61]:
checkpoint = torch.load(args_refine)
newmodel = models.__dict__[args_arch](dataset=args_dataset, depth=args_depth, cfg=checkpoint['cfg'])
newmodel.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [56]:
start_mask = torch.ones(3)
layer_id_in_cfg = 0
conv_count = 1

In [57]:
start_mask

tensor([1., 1., 1.])

In [127]:
i=0
start_mask = torch.ones(3)
layer_id_in_cfg = 0
conv_count = 1
for [m0, m1, (name,layer)] in zip(model.modules(), newmodel.modules(),  (model.named_modules())):
    if isinstance(m0, nn.Conv2d):
        if conv_count == 1:
            print('1', name)
            m1.weight.data = m0.weight.data.clone()
            conv_count += 1
            continue
        if conv_count % 2 == 0:
            print('2', name)
            mask = cfg_mask[layer_id_in_cfg]
#             print(mask)
            idx = np.squeeze(np.argwhere(np.asarray(mask.cpu().numpy())))
#             print(idx)
            w = m0.weight.data[idx.tolist(), :, :, :].clone()
#             print(m0.weight.data[idx.tolist()].shape)
            m1.weight.data = w.clone()
            layer_id_in_cfg += 1 #+
            conv_count += 1
            continue
        if conv_count % 2 == 1:
            print('3', name)
            mask = cfg_mask[layer_id_in_cfg-1]
            print(mask)
            conv_count += 1
            continue
    i+=1
    if i == 20:
        break

1 conv1
2 layer1.0.conv1
3 layer1.0.conv2
tensor([0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.])
2 layer1.1.conv1
3 layer1.1.conv2
tensor([1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.])
2 layer1.2.conv1
3 layer1.2.conv2
tensor([0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.])
2 layer1.3.conv1


IndexError: list index out of range

In [84]:
idx0 = [0]
idx1 = np.resize(idx0, (1,))
idx1

array([0])

In [81]:
idx0

array([0])